<a href="https://colab.research.google.com/github/ReynaQuita/NLP/blob/main/mbart50_wo_en2zh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets transformers sacrebleu sentencepiece jieba

     |████████████████████████████████| 1.2MB 24.8MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import pandas as pd

In [4]:
# df = pd.read_csv('/content/gdrive/MyDrive/ccmt2019-news.en2zh/ccmt2019-news.en2zh.en.txt', delimiter = "\t" )
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ccmt2019-newsen2zhsource.txt', delimiter = "\t" )
df_en = df.rename(columns={"I really don't like when people can't be themselves because they feel ... threatened, Discua says.": 'en'})
# df2 = pd.read_csv('/content/gdrive/MyDrive/ccmt2019-news.en2zh/ccmt2019-news.en2zh.zh.txt', delimiter = "\t" )
df2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ccmt2019-news.en2zh.ref1.txt', delimiter = "\t" )
df_zh = df2.rename(columns={"“我真的不喜欢人们因为感觉……受到威胁而不能做自己，”迪斯库说。": 'zh'})
df_en_zh = pd.concat([df_en, df_zh], axis=1)
dictionaryObject = df_en_zh.to_dict('record')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [5]:
dictionaryObject[:3]

[{'en': 'Court packing is integral to any effort to restore democracy, asserts Aaron Belkin, a San Francisco State University political science professor who is executive director of Pack the Courts, a group advocating an expanded bench.',
  'zh': '“法院填塞对于恢复民主是不可或缺的，”旧金山州立大学政治学教授亚伦·贝尔金说，他是“法院填塞”计划的执行理事，该组织主张扩大替补席。'},
 {'en': 'Two others are a good fit: unemployment and access to social safety nets.',
  'zh': '另外两方面非常匹配：即失业与获得社会保障。'},
 {'en': 'I really wanted to win that race so much that when I did it in the last few meters, I could not believe it myself!',
  'zh': '我真的很想赢得那场比赛，以至于当我跑到还有最后几米的时候，我自己都不敢相信！'}]

In [6]:
from datasets import load_dataset
from datasets import Dataset

x_dic={}
x_dic['translation'] = dictionaryObject
dff = pd.DataFrame( data = x_dic )
dff = dff[:50]
test_data = Dataset.from_dict( dff )
test_data

Dataset({
    features: ['translation'],
    num_rows: 50
})

In [7]:
import datasets

metric = datasets.load_metric("sacrebleu")

In [8]:
model_checkpoint = "facebook/mbart-large-50-many-to-many-mmt"

In [9]:
from transformers import AutoTokenizer, MBart50TokenizerFast
    
tokenizer = MBart50TokenizerFast.from_pretrained(model_checkpoint)

In [ ]:
# ! pip install sentencepiece

In [10]:
#from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

#model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

from transformers import MBartForConditionalGeneration

model = MBartForConditionalGeneration.from_pretrained(model_checkpoint)

In [ ]:
# if "mbart" in model_checkpoint:
#     tokenizer.src_lang = "en_XX"
#     tokenizer.tgt_lang = "zh_CN"

In [11]:
def generate_translation(batch):
    # cut off at BERT max length 512

#    english = ["translate english to chinese: " + ex["en"] for ex in batch["translation"]]
    english = [ex["en"] for ex in batch["translation"]]

    inputs = tokenizer(english, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids

    attention_mask = inputs.attention_mask

    outputs = model.generate(input_ids, attention_mask=attention_mask, forced_bos_token_id = tokenizer.lang_code_to_id["zh_CN"])
 #   outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_translation"] = output_str

    return batch

In [12]:
batch_size = 4 # change to 64 for full evaluation

results = test_data.map(generate_translation, batched=True, batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [31]:
import jieba
import nltk.translate.bleu_score as bleu

In [29]:
bleu_score = 0

for i in range(len(results)):
  print(i)
  ref = results["translation"][i]["zh"]
  pred = results["pred_translation"][i]

  ref_segment = jieba.lcut(ref)
  pred_segment = jieba.lcut(pred) 
  print('Real Translation Segmentation: {}'.format(ref_segment))
  print('Prediction Translation Segmentation: {}'.format(pred_segment))
  # print(ref_segment)
  # print(pred_segment)

  bleu_ = bleu.sentence_bleu([ref_segment], pred_segment,weights=(0.25, 0.25, 0.25, 0.25)) * 100
  bleu_score += bleu_
  print("BLEU Score: ", bleu_)
  print("-------" * 20)

0
Real Translation Segmentation: ['“', '法院', '填塞', '对于', '恢复', '民主', '是', '不可或缺', '的', '，', '”', '旧金山', '州立大学', '政治学', '教授', '亚伦', '·', '贝尔金', '说', '，', '他', '是', '“', '法院', '填塞', '”', '计划', '的', '执行', '理事', '，', '该', '组织', '主张', '扩大', '替补席', '。']
Prediction Translation Segmentation: ['旧金山', '州立大学', '政治', '科学', '教授', 'Aaron', ' ', 'Belkin', '(', 'Aaron', ' ', 'Belkin', ')', '声称', ',', '法院', '包办', '是', '恢复', '民主', '的', '任何', '努力', '中', '不可分割', '的', '一部分', '。']
BLEU Score:  29.243705060308383
--------------------------------------------------------------------------------------------------------------------------------------------
1
Real Translation Segmentation: ['另外', '两', '方面', '非常', '匹配', '：', '即', '失业', '与', '获得', '社会保障', '。']
Prediction Translation Segmentation: ['其他', '两个', '是', '合适', '的', ':', '失业', '和', '获得', '社会', '安全网', '的', '机会', '。']
BLEU Score:  68.03749333171201
----------------------------------------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [30]:
print('Bleu Score: {} (scale 0-100)'.format(bleu_score/len(results)))

Bleu Score: 30.987364903822552 (scale 0-100)


In [13]:
results['translation'][:5]

[{'en': 'Court packing is integral to any effort to restore democracy, asserts Aaron Belkin, a San Francisco State University political science professor who is executive director of Pack the Courts, a group advocating an expanded bench.',
  'zh': '“法院填塞对于恢复民主是不可或缺的，”旧金山州立大学政治学教授亚伦·贝尔金说，他是“法院填塞”计划的执行理事，该组织主张扩大替补席。'},
 {'en': 'Two others are a good fit: unemployment and access to social safety nets.',
  'zh': '另外两方面非常匹配：即失业与获得社会保障。'},
 {'en': 'I really wanted to win that race so much that when I did it in the last few meters, I could not believe it myself!',
  'zh': '我真的很想赢得那场比赛，以至于当我跑到还有最后几米的时候，我自己都不敢相信！'},
 {'en': 'Like those who follow English club Tottenham Hotspur, this was seen as a response toward rival fans who used anti-Semitism to attack the Ajax support.',
  'zh': '就像那些追随英国托特纳姆热刺队的球迷一样，这被看作是对使用反犹太主义攻击支持阿贾克斯的对手球迷的回应。'},
 {'en': 'The adoption coordinator told him they had a lot in common and both needed to work on the same things.',
  'zh': '领养协调员告诉他，他们有很多共同之处，都需要进行同样的工作。'}]

In [14]:
results['pred_translation'][:5]

['旧金山州立大学政治科学教授Aaron Belkin(Aaron Belkin)声称,法院包办是恢复民主的任何努力中不可分割的一部分。',
 '其他两个是合适的:失业和获得社会安全网的机会。',
 '我真想赢得那场比赛,当我在最后几米跑的时候,我真不敢相信!',
 '与英国俱乐部 Tottenham Hotspur的追随者一样,这被视为对使用反犹主义攻击阿贾克斯支持的对手球迷的反应。',
 '收养协调员告诉他,他们有很多共同之处,他们都需要在同样的事情上工作。']

In [18]:
one = jieba.lcut('我真想赢得那场比赛,当我在最后几米跑的时候,我真不敢相信!')
one

['我',
 '真想',
 '赢得',
 '那场',
 '比赛',
 ',',
 '当',
 '我',
 '在',
 '最后',
 '几米',
 '跑',
 '的',
 '时候',
 ',',
 '我',
 '真',
 '不敢相信',
 '!']